In [3]:
using Pkg
using IJulia
using DifferentialEquations
using Plots, CSV, XLSX, DataFrames
using Peaks, Statistics,Roots, NaNStatistics, Random
using StaticArrays
using IterTools
using Printf
using XLSX
using DataFrames

In [4]:
function freeR(rt,at,ks)
    return (rt .- at .- ks .+ sqrt.((at .- rt .- ks).^2 .+ 4*at*ks))/2
end
function freeA(rt,at,ks)
    return (at .- rt .- ks .+ sqrt.((at .- rt .- ks).^2 .+ 4*at*ks))/2
end
function compRA(rt,at,ks)
    return (at .+ rt .+ ks .- sqrt.((at .- rt .- ks).^2 .+ 4*at*ks))/2
end
function coeffI(rt,at,ka,ks,kb,kd)
    return (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)) ./ (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)*(ks*kd/(ka*kb)))
end
function coeffJ(rt,at,ka,ks,kb,kd)
    return (ks .+ ka .+ freeR(rt,at,ks)) ./ (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)*(ks*kd/(ka*kb)))
end
function tranBSD(rt,at,ka,ks,kb,kd)
    return (coeffI(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) ./ (1 .+ (coeffI(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) .+ (coeffJ(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) .* (freeR(rt,at,ks) ./ kb))
end
function rhythm(t,amp,p,ph,lev)
    rate = (amp/2)*cos(2*pi*(t-ph)/p)+lev
    return rate
end
function modelDro!(dp,p,pa,t)
    (a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd) = pa
    dp[1] = tranBSD(p[3],AT,Ka,Ks,Kb,Kd)-b1*p[1]
    dp[2] = p[1]-b2*p[2]-a3*p[2]
    dp[3] = a3*p[2]-b3*p[3]
end
function modelWT!(dp,p,pa,t)
    (a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd,ampD,phD,ampT,phT) = pa
    dp[1] = tranBSD(p[3],AT,Ka,Ks,Kb,Kd)-rhythm(t,ampD,24,phD,b1)*p[1]                       # Decay
    dp[2] = rhythm(t,ampT,24,phT,1)*p[1]-b2*p[2]-a3*p[2]                                   # Translation
    dp[3] = a3*p[2]-b3*p[3]
end

# Function to format arrays to 4 decimal places
function format_array(arr)
    return [round(x, digits=4) for x in arr]
end

# Function to compute Coefficient of Variation (CV)
function cv(x)
    return std(x) / mean(x)
end

# Define the plotting function
function create_combined_plot(detWT, timept, wtMper, wtPER, bound, wtMperScale, wtPerScale)
    # Create the first subplot
    p1 = plot([0:0.01:bound], detWT[1, 90001:(90001+bound*100)]/wtMperScale, 
        c=:green, label="Sim", linewidth=3)
    plot!(p1, timept, wtMper, seriestype=:scatter, c=:red, label="Exp (dots)", legend=:bottomleft)
    xlims!(p1, 0, bound)
    ylims!(p1, 0, 1.2)
    title!(p1, "WT_mPer")

    # Create the second subplot
    p2 = plot([0:0.01:bound], detWT[3, 90001:(90001+bound*100)]/wtPerScale, 
        c=:green, label="Sim", linewidth=3)
    plot!(p2, timept, wtPER, seriestype=:scatter, c=:red, label="Exp (dots)", legend=:bottomleft)
    xlims!(p2, 0, bound)
    ylims!(p2, 0, 1.2)
    title!(p2, "WT_PER")

    # Combine the plots
    return plot(p1, p2, layout = (2, 1), size = (1200, 1200))
end

function find_significant_extrema(data, window_size=100)
    # Find all peaks and troughs
    peaks, _ = findmaxima(data)
    troughs, _ = findminima(data)
    # Filter significant peaks and troughs based on the surrounding window
    significant_peaks = filter(p -> is_significant_extremum(data, p, window_size, >), peaks)
    significant_troughs = filter(t -> is_significant_extremum(data, t, window_size, <), troughs)
    return significant_peaks, significant_troughs
end
function is_significant_extremum(data, index, window_size, compare_func)
    start_idx = max(1, index - window_size)
    end_idx = min(length(data), index + window_size)
    # Check if the current extremum is the most extreme value in the window
    return all(i -> i == index || compare_func(data[index], data[i]), start_idx:end_idx)
end
function analyze_oscillations(time, data, n_cycles=10)
    # Use find_significant_extrema to get only significant peaks and troughs
    significant_peaks, significant_troughs = find_significant_extrema(data)
    # Combine and sort all significant extrema chronologically
    extrema = sort(vcat(significant_peaks, significant_troughs))
    # Identify whether each extremum is a peak or trough
    is_peak = [i in significant_peaks for i in extrema]
    # Initialize arrays
    peak_values = Float64[]
    trough_values = Float64[]
    peak_times = Float64[]
    trough_times = Float64[]
    amplitudes = Float64[]
    periods = Float64[]
    # Track the last peak for period calculation
    last_peak_time = nothing
    # Analyze cycles
    cycle_count = 0
    for (i, idx) in enumerate(extrema)
        if cycle_count >= n_cycles
            break
        end
        if is_peak[i]
            push!(peak_values, data[idx])
            push!(peak_times, time[idx]) 
            if !isempty(trough_values)
                # Calculate amplitude
                push!(amplitudes, data[idx] - trough_values[end])
            end          
            if !isnothing(last_peak_time)
                # Calculate period
                push!(periods, time[idx] - last_peak_time)
                cycle_count += 1
            end
            last_peak_time = time[idx]
        else
            push!(trough_values, data[idx])
            push!(trough_times, time[idx])
        end
    end
    return peak_values, trough_values, peak_times, trough_times, amplitudes, periods
end

function calculate_pcm_scaling_factor(paInitPcm)
    a3, b1, b2, b3, AT, Ka, Ks, Kb, Kd = paInitPcm

    time_range = 0:0.01:300
    bound = 300

    # Solve the ODE problem
    prob = ODEProblem(modelDro!, zeros(3), (0.0, 1200.0), paInitPcm)
    detDro = solve(prob, saveat=0.01, Rosenbrock23())
        
    # Use views instead of slices
    dp1_solution_pcm = @view detDro[1, 90001:(90001+bound*100)]
    dp3_solution_pcm = @view detDro[3, 90001:(90001+bound*100)]


    # Find peaks and troughs in one pass
    peaks_dp1, troughs_dp1 = find_significant_extrema(dp1_solution_pcm)
    peaks_dp3, troughs_dp3 = find_significant_extrema(dp3_solution_pcm)

    dp1_cycle_num = length(peaks_dp1)
    dp3_cycle_num = length(peaks_dp3)

    # Analyze oscillations (1st)
    peaks_dp1, troughs_dp1, peak_times_dp1, trough_times_dp1, amplitudes_dp1, periods_dp1 = 
        analyze_oscillations(time_range, dp1_solution_pcm, dp1_cycle_num)
    peaks_dp3, troughs_dp3, peak_times_dp3, trough_times_dp3, amplitudes_dp3, periods_dp3 = 
        analyze_oscillations(time_range, dp3_solution_pcm, dp3_cycle_num)

    # Find scaling factor 
    dp1_div_factor = !isempty(peaks_dp1) ? maximum(peaks_dp1) : 1.0
    dp3_div_factor = !isempty(peaks_dp3) ? maximum(peaks_dp3) : 1.0

    return dp1_div_factor, dp3_div_factor
end



calculate_pcm_scaling_factor (generic function with 1 method)

## pcm 파라미터 csv 파일 불러오기

In [5]:
timept = [0, 4, 8, 12, 16, 20]
wtMper = [34.845, 26.06667, 26.09167, 49.225, 55.625, 64.27083]
pcmMper = [54.59242, 57.97024, 39.78646, 37.68333, 38.33725, 47.00128]
normMper = maximum(pcmMper)
wtMper = wtMper/normMper
pcmMper = pcmMper/normMper

wtPER = [788.8639, 558.5368, 401.4341, 330.7683, 308.5475, 336.6191]
pcmPER = [1180.3421, 1028.4309, 1521.4779, 1305.6787, 895.0438, 821.742]
normPER = maximum(pcmPER)
wtPER = wtPER/normPER
pcmPER = pcmPER/normPER

# 정 박사님 csv 파일 경로
csv_file_path = "merged_final.csv"
df = CSV.File(csv_file_path) |> DataFrame

if all(ismissing, df[:, end]) 
    select!(df, Not(names(df)[end]))  
end

Row,a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.25916,0.0224804,0.121116,0.941907,0.0941511,5.18537e-5,3.00625e-9,7.67094e-5,20.6288
2,1.47946,0.0221391,0.0121071,1.42949,0.0502401,1.16233e-6,1.33764e-9,1.01303e-5,102.646
3,1.32136,0.0237089,0.0609315,0.956587,0.102341,9.87505e-5,5.62158e-9,3.90741e-7,0.860377
4,1.0953,0.0238687,0.000301884,1.1609,0.115926,2.28001e-5,8.83072e-9,1.28284e-5,6.23468
5,1.4988,0.0240382,0.0421148,1.24659,0.0769291,3.38086e-6,2.49011e-9,6.18838e-7,0.225401
6,1.63063,0.0217855,0.248852,1.28258,0.0408895,2.60133e-6,1.09041e-9,6.73333e-9,0.060827
7,1.12788,0.0227179,0.109184,0.943745,0.0978216,6.47752e-5,6.67328e-9,0.000238097,1529.04
8,1.24052,0.0241942,0.00720652,0.928571,0.0725202,6.14336e-7,1.27166e-8,2.87731e-7,0.00284178
9,1.127,0.0252269,0.034476,0.895687,0.149448,6.90149e-7,6.33369e-10,3.28417e-6,3.55003e-5


## Amplitude, Period CV 계산하는 함수

In [6]:
function calculate_amp_CV_period_CV_v3(paInit, wtMper, wtPER, excel_file_name)        
    
    a3, b1, b2, b3, AT, Ka, Ks, Kb, Kd, ampD, phD, ampT, phT = paInit

    time_range = 0:0.01:300
    bound = 300
    timept = collect(0:4:bound)

    # Solve the ODE problem
    prob = ODEProblem(modelWT!, zeros(3), (0.0, 1200.0), paInit)
    detWT = solve(prob, saveat=0.01, Rosenbrock23())

    # Calculate pcm_scaling_factor
    wtMperScale, wtPerScale = calculate_pcm_scaling_factor(paInit[1:9])                 
    detWT[1] ./ wtMperScale 
    detWT[3] ./ wtPerScale

    # Use views instead of slices
    dp1_solution = @view detWT[1, 90001:(90001+bound*100)]
    dp3_solution = @view detWT[3, 90001:(90001+bound*100)]

    # Find number of peaks and troughs in one pass
    peaks_dp1, troughs_dp1 = find_significant_extrema(dp1_solution)
    peaks_dp3, troughs_dp3 = find_significant_extrema(dp3_solution)
    dp1_cycle_num = length(peaks_dp1)
    dp3_cycle_num = length(peaks_dp3)
    
    # Analyze oscillations
    peaks_dp1, troughs_dp1, peak_times_dp1, trough_times_dp1, amplitudes_dp1, periods_dp1 = 
        analyze_oscillations(time_range, dp1_solution, dp1_cycle_num)
    peaks_dp3, troughs_dp3, peak_times_dp3, trough_times_dp3, amplitudes_dp3, periods_dp3 = 
        analyze_oscillations(time_range, dp3_solution, dp3_cycle_num)

    # Check if lengths match before division
    len_dp1 = min(length(amplitudes_dp1), length(peaks_dp1))
    len_dp3 = min(length(amplitudes_dp3), length(peaks_dp3))
    
    # Truncate arrays to the minimum length
    # Compute relamp using vectorized operations
    relamp_1 = amplitudes_dp1[1:len_dp1] ./ peaks_dp1[1:len_dp1]
    relamp_3 = amplitudes_dp3[1:len_dp3] ./ peaks_dp3[1:len_dp3]

    periods_wtMper = format_array(periods_dp1) 
    periods_wtPER = format_array(periods_dp3)

    # Calculate the period cost
    cost_periods_wtMper = sqrt(sum((1 .- periods_wtMper ./ 24).^2))    
    cost_periods_wtPER = sqrt(sum((1 .- periods_wtPER ./ 24).^2))

    cv_relamp_1 = cv(relamp_1)
    cv_relamp_3 = cv(relamp_3)
    cv_periods_wtMper = cv(periods_wtMper)
    cv_periods_wtPER = cv(periods_wtPER)
    
    # Check if the various costs are less than their threshold values
    if cv_relamp_1 < 0.1 && cv_relamp_3 < 0.1 && cv_periods_wtMper < 0.1 && cv_periods_wtPER < 0.1 && cost_periods_wtMper < 0.1 && cost_periods_wtPER < 0.1         
        # Prepare data to be saved
        data_in_row = [a3, b1, b2, b3, AT, Ka, Ks, Kb, Kd, ampD, phD, ampT, phT, 
                cv_relamp_1, cv_relamp_3, cv_periods_wtMper, cv_periods_wtPER, 
                cost_periods_wtMper, cost_periods_wtPER]

        last_row = nothing

        # Excel 파일을 열기
        XLSX.openxlsx(excel_file_name, mode="rw") do xf
            # 첫 번째 sheet 가져오기
            sheet = xf[1]
        
            # 기존 데이터의 마지막 줄 찾기
            last_row = 1
            while !ismissing(sheet[last_row, 1])  # 첫 번째 컬럼이 비어 있지 않은 줄을 찾음
                last_row += 1
            end          
            sheet[last_row, 1:length(data_in_row)] = data_in_row
        end
    end
end



calculate_amp_CV_period_CV_v3 (generic function with 1 method)

## 리듬 만드는 파라미터를 Excel 파일에 기록하는 부분

In [5]:
wtMper = [0.6010842804859873, 0.44965606490502713, 0.4500873206666041, 0.8491425945450632, 0.9595440695087687, 1.108686629553371]
wtPER = [0.5184852832893596, 0.36710148731046305, 0.26384484454227036, 0.2173993457282554, 0.2027945985939066, 0.22124481729244966]

batch_row_size = 35

for row_index in 72:(72+batch_row_size) # row: 3, 4, 5, ... , 10
    
    excel_file_name = "20241005_merged_final_v17_row_$row_index.xlsx"
    
    # 만약 excel 파일이 존재하면 삭제
    if isfile(excel_file_name)
        rm(excel_file_name)
        println("기존 파일을 삭제했습니다: $excel_file_name", "\n")
    end
    
    XLSX.openxlsx(excel_file_name, mode="w") do xf
        sheet = xf[1]  # 기본 시트 (sheet1)
        sheet[1, 1:19] = ["a3", "b1", "b2", "b3", "AT", "Ka", "Ks", "Kb", "Kd", "ampD", "phD", "ampT", "phT", "cv_relamp_1", "cv_relamp_3", "cv_periods_wtMper", "cv_periods_wtPER", "cost_periods_wtMper", "cost_periods_wtPER"]
    end
    
    """    
    ampD = [0.001] 
    phD = [0, 6, 12, 18]
    ampT = [0.001]
    phT = [0, 6, 12, 18]
    """
    # Row i의 b1 파악
    b1_at_row_index = collect(df[row_index, :])[2]
    println("row_index: ", row_index, "  ", "b1_at_row_index: ", b1_at_row_index, "\n")

    # wt Row i 후보 list 제작
    ampD = [(2*b1_at_row_index)*i / 5 for i in 1:5]      # levD = b1
    phD = [0, 6, 12, 18]
    ampT = [(2*1)*i / 5 for i in 1:5]       # LevT = 1
    phT = [0, 6, 12, 18]
    
    
    # WT 파라미터 후보값 list 기반해서, 400개 조합 생성. 
    permutations = collect(IterTools.product(ampD, phD, ampT, phT))
    
    # Calculate total number of permutations
    total_permutations = length(permutations)
    println("Total permutations to attempt: $total_permutations", "\n")

    # Flatten the 4-dimensional array into a 1-dimensional array
    permutations_flattened = reshape(permutations, :)

    # pcm9 = row i의 pcm 9개 
    row_data = collect(df[row_index, :])  # DataFrameRow를 배열로 변환
    
    i = 0
    for params in permutations_flattened
        ampD, phD, ampT, phT = params
    
        wtParam = vcat(row_data, [ampD, phD, ampT, phT])
    
        calculate_amp_CV_period_CV_v3(wtParam, wtMper, wtPER, excel_file_name)
    
        # Track progress and print every 300 iterations
        i += 1
        if i % 100 == 0
            percent_complete = round(i / total_permutations * 100, digits=2)
            println("Progress: $i / $total_permutations total permutations attempted ($(percent_complete)% completed)", "\n")
        end
    end
end

 

row_index: 72  b1_at_row_index: 0.022340568

Total permutations to attempt: 400

Progress: 100 / 400 total permutations attempted (25.0% completed)

Progress: 200 / 400 total permutations attempted (50.0% completed)

Progress: 300 / 400 total permutations attempted (75.0% completed)

Progress: 400 / 400 total permutations attempted (100.0% completed)

row_index: 73  b1_at_row_index: 0.024554115

Total permutations to attempt: 400

Progress: 100 / 400 total permutations attempted (25.0% completed)

Progress: 200 / 400 total permutations attempted (50.0% completed)

Progress: 300 / 400 total permutations attempted (75.0% completed)

Progress: 400 / 400 total permutations attempted (100.0% completed)

row_index: 74  b1_at_row_index: 0.022451746312090327

Total permutations to attempt: 400

Progress: 100 / 400 total permutations attempted (25.0% completed)

Progress: 200 / 400 total permutations attempted (50.0% completed)

Progress: 300 / 400 total permutations attempted (75.0% completed)